# Capstone Project - The Battle of Neighborhoods (Week 1)

## Introduction / Business Problem

When visiting a major city in another country for a short period, it can be difficult to establish the orientation and location of interesting  “neighbourhoods”.  Unless aided by friends or family who know the destination city well, it is common to get stuck in the tourist quarters, and miss the vibrancy of each city’s more interesting parts. In London, where I live, tourists are rarely seen outside of relatively few and relatively central locations: Westminster, the West End, the South Bank, and Shoreditch.  This could be partly due to the contrivances of guide books and partly due to the overwhelming size of modern cities. 

To get a sense of what each neighbourhood in a destination city is like, it can be useful to “translate” their characteristics to a more familiar place. For example, Peckham and Brooklyn are more similar than, say, Peckham and Midtown, which would be a helpful orientation device for a native Londoner (particularly if they are in search of craft beer and hipsters).

A similar problem is contemplated by the site Hoodmaps ([https://hoodmaps.com](https://hoodmaps.com/)), which relies on a crowdsourcing technique to characterise the neighbourhoods of cities around the world.  This is a really interesting approach, however, I propose a different approach, leveraging existing location data, such as venue-related data offered via the Foursquare API, which would be interesting to tourists.  By building a venue-based profile for each neighbourhood in a number of cities, it would be possible to employ a clustering approach across cities to identify similarities in neighbourhoods based on the venue types located there.

To test this concept, I will use three major cities, about which there will be sufficiently large datasets available: London, New York, and Paris.  I have selected these cities, as I have first-hand experience of spending significant periods of time in each, and can compare the results with the “ground truth”, given that clustering will be unsupervised without a formal evaluation technique.